In [39]:
import pandas as pd
import numpy as np
import math
import sqlite3
import json
import ast

In [66]:
lis_months = ["F","G","H","J","K","M","N","Q","U","V","X","Z"]
month_to_index = {m: i for i, m in enumerate(lis_months)}


In [35]:
def expand_contracts_and_lots(contracts, lots):
    if len(contracts) != len(lots):
        raise ValueError("contracts and lots must have same length")

    # ---- Parse contracts into (month_index, year) ----
    parsed = []
    for c in contracts:
        month = c[0]
        year = int(c[1:])  # last two digits
        parsed.append((month_to_index[month], year))

    # ---- Determine the start and end points ----
    start_month, start_year = parsed[0]
    end_month, end_year = parsed[-1]

    # ---- Expand continuously month-by-month ----
    expanded_contracts = []
    expanded_lots = []

    lot_map = {contracts[i]: lots[i] for i in range(len(contracts))}

    ym = start_month
    yy = start_year

    while True:
        # Make contract code
        contract_code = lis_months[ym] + str(yy)

        # append contract
        expanded_contracts.append(contract_code)

        # Append lot if exists, else 0
        expanded_lots.append(lot_map.get(contract_code, 0))

        # Stop condition
        if yy == end_year and ym == end_month:
            break

        # Increment month
        ym += 1
        if ym == 12:
            ym = 0
            yy += 1

    return expanded_contracts, expanded_lots

In [69]:
def next_contract(contract, step):
    month = contract[0]              # e.g. "H"
    year = int(contract[1:])         # e.g. "25"

    idx = month_to_index[month]         # find index of "H"
    new_idx = idx + step             # add step

    # adjust year when crossing December → January
    year += new_idx // 12            # integer rollover
    new_month = lis_months[new_idx % 12]

    return f"{new_month}{year}"

In [70]:
next_contract('H25', 1)

'J25'

In [5]:
a = 's'
if not a:
    print('no')

In [22]:
def create_custom_strategy_intraproduct(strategy_name, lis_lots):

    # ---- 1. Factorize lots ----
    non_zero_positive_sizes = [abs(x) for x in lis_lots if x != 0]

    if not non_zero_positive_sizes:
        raise ValueError("Lots list cannot have all zeros.")

    gcd = non_zero_positive_sizes[0]
    for n in non_zero_positive_sizes[1:]:
        gcd = math.gcd(gcd, n)

    lots_lis_factorized = [x // gcd for x in lis_lots]

    # ---- 2. Save to DB ----
    conn = sqlite3.connect("positionmanager.db")
    cur = conn.cursor()

    # Create table if not exists
    cur.execute("""
        CREATE TABLE IF NOT EXISTS custom_strategies (
            structure_name TEXT PRIMARY KEY,
            structure_lots TEXT
        )
    """)

    # Convert list → JSON for safe storage
    lots_json = json.dumps(lots_lis_factorized)

    # Insert or replace (overwrite if strategy name already exists)
    cur.execute("""
        INSERT OR REPLACE INTO custom_strategies (structure_name, structure_lots)
        VALUES (?, ?)
    """, (strategy_name, lots_json))

    conn.commit()
    conn.close()



In [48]:
def hedge_outrights(product, lis_contracts, lis_lots):
    expanded_lis_contracts, expanded_lis_lots = expand_contracts_and_lots(lis_contracts, lis_lots)
    # ---- 1. Factorize lots ----
    non_zero_positive_sizes = [abs(x) for x in expanded_lis_lots if x != 0]

    if not non_zero_positive_sizes:
        raise ValueError("Lots list cannot have all zeros.")

    gcd = non_zero_positive_sizes[0]
    for n in non_zero_positive_sizes[1:]:
        gcd = math.gcd(gcd, n)

    lis_lots_factorized = [x // gcd for x in expanded_lis_lots]

# ---- 3. Fetch matching strategy from DB ----
    conn = sqlite3.connect("positionmanager.db")
    cursor = conn.cursor()

    cursor.execute("SELECT structure_name, structure_lots FROM custom_strategies")
    rows = cursor.fetchall()

    structure_name = None

    for name, lots_str in rows:
        stored_lots = ast.literal_eval(lots_str)   # safe parse "[1,-2,1]"
        if stored_lots == lis_lots_factorized:
            structure_name = name
            break

    conn.close()

    if structure_name is None:
        raise ValueError("No matching custom strategy found for factorized lots.")

    # ---- 4. Compute lots = lis_contracts[0] / lots_lis_factorized[0] ----
    first_lots_factorized = lis_lots_factorized[0]
    first_lots = lis_lots[0]

    num_lots = first_lots // first_lots_factorized

    return {
        "structure_name": structure_name,
        "lots": num_lots
    }
    

In [71]:
def unhedge_structure_into_outrights(structure_name, starting_contract, num_lots):
    """
    Convert a structure name + number of lots → outright contracts + lots per contract.

    Example:
        structure_name = '1 mo butterfly'
        starting_contract = 'H25'
        num_lots = 5

        → ['H25','J25','K25'], [5, -10, 5]
    """

    # ---- 1. Get lots pattern from DB ----
    conn = sqlite3.connect("positionmanager.db")
    cur = conn.cursor()
    cur.execute("""
        SELECT structure_lots FROM custom_strategies WHERE structure_name = ?
    """, (structure_name,))
    row = cur.fetchone()

    if row is None:
        raise ValueError(f"Strategy '{structure_name}' not found in custom_strategies.")

    lots_pattern = ast.literal_eval(row[0])   # e.g. [1, -2, 1]
    n = len(lots_pattern)

    # ---- 2. Generate contracts from starting_contract ----
    lis_contracts = []
    for i in range(n):
        c = next_contract(starting_contract, i)
        lis_contracts.append(c)

    # ---- 3. Scale lots by num_lots ----
    lis_lots_per_contract = [x * num_lots for x in lots_pattern]

    return lis_contracts, lis_lots_per_contract

In [ ]:
def implement_hedge()

In [55]:
strategy_name, lis_lots = '1 mo butterfly', [5,-10,5]

create_custom_strategy_intraproduct(strategy_name, lis_lots)


In [51]:
product, lis_contracts, lis_lots = 'CL', ['H25', 'J25', 'K25'], [5,-10,5]
hedge_outrights(product, lis_contracts, lis_lots)

{'structure_name': '1 mo butterfly', 'lots': 5}

In [72]:
product, lis_contracts, lis_lots = 'CL', ['H25', 'J25', 'K25'], [5,-10,5]
unhedge_structure_into_outrights('1 mo butterfly', 'H25', 5)

(['H25', 'J25', 'K25'], [5, -10, 5])

In [20]:
conn = sqlite3.connect("positionmanager.db")
df = pd.read_sql_query("SELECT * FROM custom_strategies", conn)
conn.close()

print(df)

   structure_name structure_lots
0  1 mo butterfly     [1, -2, 1]
